In [5]:
import torch
from models import BiTLikeModel

# Load the model checkpoint
checkpoint = torch.load('best.pt', map_location=torch.device('cpu'))

# Initialize the model
model = BiTLikeModel(2)  # Replace with your model class

# Get the model's state_dict
model_state_dict = model.state_dict()

# Filter out mismatched keys (e.g., fc layer)
filtered_checkpoint = {k: v for k, v in checkpoint.items() if k in model_state_dict and model_state_dict[k].shape == v.shape}

# Load the compatible weights
model_state_dict.update(filtered_checkpoint)
model.load_state_dict(model_state_dict)

# Reinitialize the fc layer
torch.nn.init.xavier_uniform_(model.fc.weight)
torch.nn.init.zeros_(model.fc.bias)

# Set the model to evaluation mode
model.eval()

print("Model loaded successfully!")

Model loaded successfully!


In [6]:
import torch
from torchvision import transforms
from PIL import Image

def predict_image_class(model, image_path):
    """
    Predicts the class of an image using the given model.

    Args:
        model (torch.nn.Module): The trained model for inference.
        image_path (str): Path to the image file.

    Returns:
        int: Predicted class index.
    """

    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match the model's input size
        transforms.ToTensor(),         # Convert image to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
    ])

    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(image).unsqueeze(0)  # Add batch dimension

    # Perform inference
    with torch.no_grad():
        output = model(input_tensor)
        print(f"output: {output}")
        probabilities = torch.softmax(output, dim=1)
        print(f"probabilities: {probabilities}")

    # Get the predicted class
    predicted_class = torch.argmax(probabilities, dim=1).item()
    print(f"predicted_class: {predicted_class}")
    return predicted_class

In [14]:
#testuojam

# atsispaudzia
image_path = 'test_data/ats1.jpeg'
predict_image_class(model, image_path)
image_path = 'test_data/ats2.jpeg'
predict_image_class(model, image_path)

print("=======================")

# stovi
image_path = 'test_data/stov1.jpeg'
predict_image_class(model, image_path)
image_path = 'test_data/stov2.jpeg'
predict_image_class(model, image_path)

output: tensor([[-0.1558, -0.0863]])
probabilities: tensor([[0.4826, 0.5174]])
predicted_class: 1
output: tensor([[-0.2289, -0.0077]])
probabilities: tensor([[0.4449, 0.5551]])
predicted_class: 1
output: tensor([[-0.0492, -0.1329]])
probabilities: tensor([[0.5209, 0.4791]])
predicted_class: 0
output: tensor([[-0.0183, -0.1362]])
probabilities: tensor([[0.5294, 0.4706]])
predicted_class: 0


0

In [7]:
import cv2
import os
import time

def capture_and_predict(model, capture_interval=5):
    """
    Automatically captures images from the webcam at regular intervals and predicts their class using the given model.

    Args:
        model (torch.nn.Module): The trained model for inference.
        capture_interval (int): Time interval (in seconds) between captures.

    Returns:
        None
    """
    # Initialize the webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise Exception("Could not open webcam")

    print("Capturing images automatically. Press 'Esc' to exit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Display the webcam feed
        cv2.imshow("Webcam", frame)

        # Save the captured frame as a temporary image
        temp_image_path = "/tmp/temp_image.jpg"
        cv2.imwrite(temp_image_path, frame)
        print("Image captured!")

        # Predict the class of the captured image
        predicted_class = predict_image_class(model, temp_image_path)
        print(f"Predicted class: {predicted_class}")

        # play bell when prediction is 1
        if predicted_class == 1:
            os.system('play -nq --null --channels 1 synth 0.1 sine 444')
        else:
            os.system('play -nq --null --channels 1 synth 0.2 sine 1888')

        # Remove the temporary image
        os.remove(temp_image_path)

        # Wait for the specified interval or until 'Esc' is pressed
        if cv2.waitKey(capture_interval * 1000) == 27:  # Esc key to exit
            break

    # Release the webcam and close the window
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
capture_and_predict(model)

Capturing images automatically. Press 'Esc' to exit.
Image captured!
output: tensor([[ 0.0187, -0.2293]])
probabilities: tensor([[0.5617, 0.4383]])
predicted_class: 0
Predicted class: 0
Image captured!
output: tensor([[ 0.0463, -0.0603]])
probabilities: tensor([[0.5266, 0.4734]])
predicted_class: 0
Predicted class: 0
Image captured!
output: tensor([[ 0.0420, -0.1515]])
probabilities: tensor([[0.5482, 0.4518]])
predicted_class: 0
Predicted class: 0
Image captured!
output: tensor([[ 0.0126, -0.1336]])
probabilities: tensor([[0.5365, 0.4635]])
predicted_class: 0
Predicted class: 0
Image captured!
output: tensor([[ 0.0127, -0.1338]])
probabilities: tensor([[0.5365, 0.4635]])
predicted_class: 0
Predicted class: 0
Image captured!
output: tensor([[ 0.0182, -0.1520]])
probabilities: tensor([[0.5425, 0.4575]])
predicted_class: 0
Predicted class: 0
Image captured!
output: tensor([[ 0.0224, -0.1560]])
probabilities: tensor([[0.5445, 0.4555]])
predicted_class: 0
Predicted class: 0
Image captured!


In [14]:
os.system('play -nq --null --channels 1 synth 0.5 sine 1840')

0

In [5]:
# Check if webcam is available
import cv2

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Could not open webcam")
else:
    print("Webcam initialized successfully")
cap.release()

Webcam initialized successfully
